<a href="https://colab.research.google.com/github/likuoping136/stable-diffusion-webui-colab/blob/main/install_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
from google.colab import drive
drive.mount('/content/drive')

%env PYTHONDONTWRITEBYTECODE=1

# 导入必要的模块用于检查路径存在性
import os
import subprocess

def check_dir_exists(directory):
    """检查目录是否存在"""
    return os.path.exists(directory) and os.path.isdir(directory)

def check_file_exists(filepath):
    """检查文件是否存在"""
    return os.path.exists(filepath) and os.path.isfile(filepath)
def run_command_realtime(command):
    # 使用Popen启动进程，将标准输出和标准错误重定向到管道
    process = subprocess.Popen(command,
                               shell=True,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,  # 将标准错误合并到标准输出
                               text=True,
                               bufsize=10240,  # 行缓冲
                               universal_newlines=True)

    # 逐行读取输出
    while True:
        # 读取一行输出
        line = process.stdout.readline()
        if line:  # 如果读取到内容
            print(line, end='')  # 实时打印到控制台，end='' 避免重复换行
        elif process.poll() is not None:  # 如果进程已结束且没有更多输出
            break

    # 获取最终返回码
    return process.poll()


def run_command(cmd, check_exists_path=None, is_dir_check=True):
    """执行命令前检查路径是否存在"""
    print(f"检查路径:{check_exists_path}")
    if check_exists_path:
        if is_dir_check:
            exists = check_dir_exists(check_exists_path)
        else:
            exists = check_file_exists(check_exists_path)

        if exists:
            print(f"路径已存在，跳过")
            return True
        else:
            print(f"执行命令: {cmd}")
            return run_command_realtime(cmd)== 0
    else:
        print(f"执行命令: {cmd}")
        return  run_command_realtime(cmd) == 0


# 主仓库克隆和更新
webui_path = "/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui"
run_command('git clone  https://github.com/likuoping136/stable-diffusion-webui.git ' + webui_path, webui_path)

file_path = '/content/drive/MyDrive/stable-diffusion-webui-colab/libtcmalloc_minimal.so.4'
if run_command('wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O '+file_path, file_path, False):
    print('下载tcmalloc 成功 设置环境变量')
    %env LD_PRELOAD=/content/drive/MyDrive/stable-diffusion-webui-colab/libtcmalloc_minimal.so.4

# 下载 run_n_times.py 脚本
run_script_path = "/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/scripts/run_n_times.py"
run_command('wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O ' + run_script_path,
            run_script_path, False)

# 扩展插件列表（名称和GitHub地址）
extensions = [
    ("deforum-for-automatic1111-webui", "https://github.com/deforum-art/deforum-for-automatic1111-webui"),
    ("stable-diffusion-webui-images-browser", "https://github.com/camenduru/stable-diffusion-webui-images-browser"),
    ("stable-diffusion-webui-huggingface", "https://github.com/camenduru/stable-diffusion-webui-huggingface"),
    ("sd-civitai-browser", "https://github.com/camenduru/sd-civitai-browser"),
    ("sd-webui-additional-networks", "https://github.com/kohya-ss/sd-webui-additional-networks"),
    ("sd-webui-controlnet", "https://github.com/Mikubill/sd-webui-controlnet"),
    ("openpose-editor", "https://github.com/fkunn1326/openpose-editor"),
    ("sd-webui-depth-lib", "https://github.com/jexom/sd-webui-depth-lib"),
    ("posex", "https://github.com/hnmr293/posex"),
    ("sd-webui-3d-open-pose-editor", "https://github.com/nonnonstop/sd-webui-3d-open-pose-editor"),
    ("sd-webui-tunnels", "https://github.com/camenduru/sd-webui-tunnels"),
    ("batchlinks-webui", "https://github.com/etherealxx/batchlinks-webui"),
    ("stable-diffusion-webui-catppuccin", "https://github.com/camenduru/stable-diffusion-webui-catppuccin"),
    ("stable-diffusion-webui-rembg", "https://github.com/likuoping136/stable-diffusion-webui-colab.git"),
    ("stable-diffusion-webui-two-shot", "https://github.com/ashen-sensored/stable-diffusion-webui-two-shot"),
    ("sd-webui-aspect-ratio-helper", "https://github.com/thomasasfk/sd-webui-aspect-ratio-helper")
]

# 克隆或更新扩展插件
extensions_dir = "/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions"

for ext_name, ext_url in extensions:
    ext_path = os.path.join(extensions_dir, ext_name)
    run_command(f'git clone {ext_url} {ext_path}', ext_path)


# HuggingFace 嵌入和Lora模型
embeddings_path = "/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/embeddings/negative"
lora_path = "/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/Lora/positive"

run_command('git clone https://huggingface.co/embed/negative ' + embeddings_path, embeddings_path)
run_command('git clone https://huggingface.co/embed/lora ' + lora_path, lora_path)

# 切换到工作目录
os.chdir(webui_path)
print(f"已切换到工作目录: {os.getcwd()}")



!apt -y install -qq  aria2

# 1. 检查并下载超分辨率模型 4x-UltraSharp.pth
esrgan_model_path = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/ESRGAN/'
esrgan_file_name = '4x-UltraSharp.pth'
esrgan_all_path = os.path.join(esrgan_model_path,esrgan_file_name)
run_command('aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d '+esrgan_model_path+' -o '+esrgan_file_name, esrgan_all_path, False)
# 2. 检查并下载 ControlNet 模型文件
controlnet_model_dir = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/sd-webui-controlnet/models'

# 定义一个列表，包含要下载的模型文件信息（URL 和 输出文件名）
controlnet_models = [
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors", "control_v11p_sd15_canny_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors", "control_v11p_sd15_lineart_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml", "control_v11p_sd15_canny_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml", "control_v11f1p_sd15_depth_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml", "control_v11p_sd15_inpaint_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml", "control_v11p_sd15_mlsd_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml", "control_v11p_sd15_normalbae_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml", "control_v11p_sd15_openpose_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml", "control_v11p_sd15_scribble_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml", "control_v11p_sd15_seg_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml", "control_v11p_sd15_softedge_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml", "control_v11p_sd15s2_lineart_anime_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml", "control_v11f1e_sd15_tile_fp16.yaml")
]

for model_url, output_file in controlnet_models:
    file_path = os.path.join(controlnet_model_dir, output_file)
    run_command('aria2c --console-log-level=error -c -x 16 -s 16 -k 1M '+model_url+' -d '+controlnet_model_dir+' -o '+output_file, file_path, False)


# 3. 检查并下载基础模型 v1-5-pruned-emaonly.ckpt
sd_model_path = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/Stable-diffusion'
run_command('aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -d '+sd_model_path+' -o v1-5-pruned-emaonly.ckpt', sd_model_path+'/v1-5-pruned-emaonly.ckpt', False)

clip_path='/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/CLIP'
run_command('aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt -d '+clip_path+' -o ViT-L-14.pt',clip_path+'/ViT-L-14.pt', False)
print("模型下载检查完毕。")




%cd /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui

!mkdir /content/drive/MyDrive/stable-diffusion-webui-colab/pip-list
piplistPath='/content/drive/MyDrive/stable-diffusion-webui-colab/pip-list'
run_command('mkdir '+piplistPath+' && pip install scikit-image==0.21.0 insightface xformers --target '+piplistPath,piplistPath)
!python launch.py --skip-torch-cuda-test --nostart

print("安装流程完成。")
